In [1]:
#IMPORTAR BIBLIOTECAS UTILIZADAS

import numpy as np
import operator
from numpy import unravel_index
import pandas as pd
import random
from collections import Counter
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
"""
    Classe para processar o dataset
    Obs: o dataset tem que ser para problemas de classificacao e nele tem que conter uma coluna Class
"""
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [3]:
#IMPORTANDO DATASET
dataset = ProcessarDados('../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv')

In [4]:
#DEFINIR FUNCAO LOGISTICA (SIGMOID) PARA PREVER A PROBABILIDADE 
def sigmoid(w,X):
    z = np.dot(X,w)
    return 1 / (1 + np.exp(-z))

#DEFINIR FUNCAO PARA CLASSIFICAR
def classificar(y, limiar=0.5):
    return (y > limiar)*1

#DEFINIR FUNCAO CUSTO UTILIZANDO ENTROPIA CRUZADA
def entropiaCruzada(previsto,y):
    return (-y * np.log(previsto) - (1 - y) * np.log(1 - previsto)).mean()

#DEFINIR FUNCAO GRADIENTE DESCENDENTE PARA ATUALIZAR OS PARAMÊTROS w
def gradienteDescendente(w, X, y, alpha):
    previsto = sigmoid(w,X)
    erro = previsto - y
    gradiente = np.dot(X.T, erro) / len(X)
    w -= alpha*gradiente

In [5]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada

def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_specificity = [] # taxa de verdadeiros negativos ou especificidade
    resultados_f2 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        #SEPARAR DADOS DE TREINO E TESTE MANTENDO A MESMA PROPORÇÃO  (80% E 20%)
        seed = i
        X_treino, X_teste, y_treino, y_teste = dataset.holdout(0.2, seed)
        
        #ALTERANDO DIMENSÃO DE Y PARA D=1
        y_treino = y_treino.reshape(480,)
        y_teste = y_teste.reshape(120,)
        
        #ADICIONAR ATRIBUTOS IGUAIS A 1 EM X0
        X_treino = np.c_[np.ones(X_treino.shape[0]), X_treino]
        X_teste = np.c_[np.ones(X_teste.shape[0]), X_teste]
        
        array_w = {}
        array_alpha = {}
        array_acc = {}
        # rodando o grid search
        for index_g, grid_alpha in enumerate([0.0001, 0.001, 0.01, 0.1, 1]):
            # rodando o k-folds
            for index_k, k in enumerate(dataset.kfolds(X_treino, y_treino, k_folds, seed)):
                kx_treino, ky_treino, kx_teste, ky_teste = k

                #INICIALIZAR W (THETA) DE FORMA RANDOMICA e OTIMIZAR PARÂMETROS 
                w = np.random.rand(kx_treino.shape[1])

                # otimização do W
                for x in range(max_iter):
                    custo = entropiaCruzada(sigmoid(w,kx_treino),ky_treino)
                    gradienteDescendente(w,kx_treino,ky_treino,grid_alpha)

                    
                #TESTANDO O MODELO
                ky_pred = classificar(sigmoid(w,kx_teste))
                k_accuracy = metrics.accuracy_score(ky_teste, ky_pred)

                array_w[index_g,index_k] = w
                array_alpha[index_g,index_k] = grid_alpha
                array_acc[index_g,index_k] = k_accuracy
            
        # acessando o melhor W pela acurácia
        pos_best_lin = max(array_acc.items(), key=operator.itemgetter(1))[0][0]
        pos_best_col = max(array_acc.items(), key=operator.itemgetter(1))[0][1]
        
        best_w = array_w[pos_best_lin,pos_best_col]
        
        
        #TESTE FINAL O MODELO COM O MELHOR W APÓS O KFOLD
        y_pred = classificar(sigmoid(best_w, X_teste))
            
        
        cm  = confusion_matrix(y_teste, y_pred)
        if exibir_matriz_confusao:
            print(cm)

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_teste, y_pred)
        precision = metrics.precision_score(y_teste, y_pred)
        recall = metrics.recall_score(y_teste, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        beta = 0.5
        f2_score = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)
        

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_specificity.append(specificity)
        resultados_f2.append(f2_score)


        best_parametros = {'alpha' : array_alpha[pos_best_lin,pos_best_col]};
        resultados_parametros.append(best_parametros)
        
        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("Specificity:",specificity)
            print("f2-Score:",f2_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['specificity'] = resultados_specificity
    resultados_gerais['f2'] = resultados_f2
    resultados_gerais['params'] = resultados_parametros
    
    return resultados_gerais

In [6]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\tSpecificity \t|\t fb-Score")
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='  ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end=' ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['specificity'], axis=0), np.std(dict_metricas['specificity'], axis=0)),end='   ')
        print ("      %.4f +- %.4f" % (np.mean(dict_metricas['f2'], axis=0), np.std(dict_metricas['f2'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

In [7]:
#DEFININDO OS PARÂMETROS
max_iter = 6000
epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True

# TREINANDO O MODELO E OBTENDO AS MÉTRICAS

dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('rl',dict_metricas)

[[24  4]
 [ 7 85]]
Rodada: # 0
{'alpha': 1}
Accuracy: 0.9083333333333333
Precision: 0.9550561797752809
Recall: 0.9239130434782609
Specificity: 0.8571428571428571
f2-Score: 0.9486607142857142


[[15 18]
 [ 3 84]]
Rodada: # 1
{'alpha': 0.001}
Accuracy: 0.825
Precision: 0.8235294117647058
Recall: 0.9655172413793104
Specificity: 0.45454545454545453
f2-Score: 0.8484848484848484


[[16  6]
 [ 9 89]]
Rodada: # 2
{'alpha': 0.1}
Accuracy: 0.875
Precision: 0.9368421052631579
Recall: 0.9081632653061225
Specificity: 0.7272727272727273
f2-Score: 0.9309623430962345


[[24  8]
 [ 4 84]]
Rodada: # 3
{'alpha': 0.1}
Accuracy: 0.9
Precision: 0.9130434782608695
Recall: 0.9545454545454546
Specificity: 0.75
f2-Score: 0.9210526315789473


[[24  4]
 [ 4 88]]
Rodada: # 4
{'alpha': 1}
Accuracy: 0.9333333333333333
Precision: 0.9565217391304348
Recall: 0.9565217391304348
Specificity: 0.8571428571428571
f2-Score: 0.9565217391304348


[[18 14]
 [ 2 86]]
Rodada: # 5
{'alpha': 0.01}
Accuracy: 0.8666666666666667
Preci

[[22 11]
 [ 4 83]]
Rodada: # 44
{'alpha': 1}
Accuracy: 0.875
Precision: 0.8829787234042553
Recall: 0.9540229885057471
Specificity: 0.6666666666666666
f2-Score: 0.896328293736501


[[18 15]
 [ 1 86]]
Rodada: # 45
{'alpha': 0.01}
Accuracy: 0.8666666666666667
Precision: 0.8514851485148515
Recall: 0.9885057471264368
Specificity: 0.5454545454545454
f2-Score: 0.875763747454175


[[21  4]
 [ 7 88]]
Rodada: # 46
{'alpha': 1}
Accuracy: 0.9083333333333333
Precision: 0.9565217391304348
Recall: 0.9263157894736842
Specificity: 0.84
f2-Score: 0.9503239740820735


[[17  7]
 [ 4 92]]
Rodada: # 47
{'alpha': 0.1}
Accuracy: 0.9083333333333333
Precision: 0.9292929292929293
Recall: 0.9583333333333334
Specificity: 0.7083333333333334
f2-Score: 0.9349593495934959


[[21 10]
 [ 2 87]]
Rodada: # 48
{'alpha': 0.1}
Accuracy: 0.9
Precision: 0.8969072164948454
Recall: 0.9775280898876404
Specificity: 0.6774193548387096
f2-Score: 0.9119496855345911


[[23 11]
 [ 1 85]]
Rodada: # 49
{'alpha': 0.01}
Accuracy: 0.9
Preci